# &#9989; Import modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyroomacoustics as pyra
import scipy.io
from scipy.io import wavfile
import IPython
import pickle
import storedata as sd
import dataSetAudio as dsa
import matrestriction as mr

## &#128190; Set parameters and retrieve signal modeling matrices.

In [3]:
#Set parameters for Beamforming
M = 8         #number of mics in the array
J = 48        #48 tap delay lines for each channel
NumAng = 181  #considering 181 DOAs in the scan of directions
    
# Get the restriction matrices for:
# -a Beamformer of 8 channels
# -48 tap delay lines for each channel
# -scan of 181 DOAs
# -fs=16000 Hz
sd.load("restriction_matrices")
C_list,fr_list,Bk_list = sd.C_list,sd.fr_list,sd.Bk_list # restriction matrices for ULA
Cc_list,frc_list,Bkc_list = sd.Cc_list,sd.frc_list,sd.Bkc_list # restriction matrices for UCA

## &#127925; &#128296; Get the audio and simulate the soundstage

In [78]:
#Get the audio files and sampling frequency
fs,data1 = wavfile.read("../voice_data/interfering_voice.wav")
_,data2 = wavfile.read("../voice_data/desired_voice.wav")

#Create an object
direcc_arrival = [24, -41]
reverb = 0.22
snr = 0
distances = [1.7, 2.1]

ex = dsa.dataSetAudio(direcc_arrival, reverb, snr, distances)
M_Sigs = ex.SimuData(_, [data1,data2], fs, M, delay=[0,0],N=2,array_mic="ULA",dataset=False)

IPython.display.Audio(M_Sigs, rate=fs)

##  &#128187; &#128640; GSC Algorithm. Enhance the resultant signal and separate sources

In [82]:
ind = mr.getIdx(direcc_arrival,NumAng, array_mic='ULA')
signal = ind[1]
matrices = (C_list[signal],fr_list[signal],Bk_list[signal])

coeficientes = np.dot(matrices[0],np.dot(np.linalg.inv(np.dot(matrices[0].T, matrices[0])),matrices[1]))
w_adp = np.zeros(Bk_list[ind[1]].shape[1])

muu = 2e-10
err = np.zeros(M_Sigs.shape[1])
Xd = np.zeros((M,J))
M_S = np.concatenate((M_Sigs[:,::-1],Xd),axis=1)
for i in range(1,M_S.shape[-1]-J):
    Xmj = M_S[:,-i-J:-i].T.flatten()
    out_up = np.dot(coeficientes,Xmj)
    x_u = np.dot(matrices[2].T,Xmj)
    err[i] = out_up - np.dot(x_u,w_adp)
    w_adp += muu*err[i]*x_u
    pass

err = 0.5*err
IPython.display.Audio(err,rate=fs)    